In [ ]:
## Idenitfy how many pixels need to be avergaed until mean converges to a steady value.

# 1. identify sample SPOTS in raster scene as polygons
# 2. mask raster to only show data within these polygonns
# 3. extract individual pixel values for given Band/wavelength
# 4. sort pixels randomly
# 5. iterativly calculate mean spectral value as pixels are added one at a time. 
# 6. identify number of pixels where mean spectral value begins to stabalize. This is the minimum number of pixels needed to charectorize a targeted surface.
# 7. Conver minimum number of pixels to minimum number of SPOT polygons at the 50-meter, 100-meter, and 150-meter radius (e.g. 9 pixels, 20 pixels, and X pixels each)